In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import timeit
from mlmodel import *
import pickle
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
import utils
from torchvision import transforms
import torchvision.models as models
from torch import nn

In [2]:
!ls ../ZSTL_Data/CUB_200_2011/CUB_200_2011/label_feature.txt

../ZSTL_Data/CUB_200_2011/CUB_200_2011/label_feature.txt


In [3]:
num_task = 200
num_data = 40

path_feature = '../ZSTL_Data/CUB_200_2011/CUB_200_2011/label_feature.txt'
path_attributes = '../ZSTL_Data/CUB_200_2011/CUB_200_2011/attributes/class_attribute_labels_continuous.txt'
path_destination = '../ZSTL_Data/CUB_200_2011/CUB_200_2011/splitedTask/'

np.random.seed(888)

data_label_feature = pd.read_csv(path_feature, sep=" ", header=None)

data_label_feature.columns = ['label'] + [x for x in range(2048)]
print(data_label_feature)

label         0         1         2         3         4         5  \
0        1.0  0.554742  0.449377  0.787002  0.377700  0.439481  0.487352   
1        1.0  0.518317  0.397836  0.776558  0.384175  0.440413  0.482135   
2        1.0  0.509601  0.438840  0.742074  0.355505  0.458852  0.520114   
3        1.0  0.527033  0.440028  0.742868  0.343865  0.448310  0.488656   
4        1.0  0.552168  0.386958  0.795093  0.359147  0.445325  0.528473   
...      ...       ...       ...       ...       ...       ...       ...   
11775  200.0  0.530580  0.413276  0.737307  0.407817  0.466526  0.536493   
11776  200.0  0.535687  0.457732  0.765419  0.369241  0.504226  0.526427   
11777  200.0  0.514037  0.482673  0.748582  0.373150  0.482410  0.495119   
11778  200.0  0.538375  0.463504  0.765660  0.379090  0.448390  0.522997   
11779  200.0  0.560232  0.476490  0.759214  0.389367  0.477937  0.500162   

              6         7         8  ...      2038      2039      2040  \
0      0.417452  0.5

In [4]:
stat = []
for r in range(num_task):
    df = data_label_feature.loc[data_label_feature['label'].eq(r+1)]
    print(df.shape)
    stat.append(df.shape[0])
print(min(stat))

(60, 2049)
(60, 2049)
(58, 2049)
(60, 2049)
(44, 2049)
(41, 2049)
(53, 2049)
(48, 2049)
(58, 2049)
(60, 2049)
(60, 2049)
(56, 2049)
(60, 2049)
(60, 2049)
(58, 2049)
(58, 2049)
(57, 2049)
(45, 2049)
(59, 2049)
(59, 2049)
(60, 2049)
(56, 2049)
(59, 2049)
(52, 2049)
(59, 2049)
(60, 2049)
(60, 2049)
(59, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(53, 2049)
(59, 2049)
(59, 2049)
(60, 2049)
(60, 2049)
(59, 2049)
(60, 2049)
(59, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(59, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(58, 2049)
(60, 2049)
(59, 2049)
(60, 2049)
(60, 2049)
(58, 2049)
(60, 2049)
(50, 2049)
(59, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(57, 2049)
(60, 2049)
(60, 2049)
(59, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(53, 2049)
(60, 2049)
(60, 2049)
(59, 2049)
(60, 2049)
(60, 2049)
(60, 2049)
(60, 2049)

In [5]:
def generate_compressed_data(data, num_task, num_data):
    # print('row')
    data_numpy = data.to_numpy()
    data_x = data_numpy[:,1:]
    data_y = np.atleast_2d(data_numpy[:,0])
    print(data_x.shape)
    start = timeit.default_timer()
    # pca = PCA(n_components=1024)
    # data_x_compress = pca.fit_transform(data_x)
    data_x_compress = data_x
    print('data_x_compress ',data_x_compress.shape, data_x_compress)
    stop = timeit.default_timer()
    print('Time:', stop - start)

    data_compress_np = np.hstack((data_y.T, data_x_compress))
    data_compressed =  pd.DataFrame(data_compress_np,
                   columns=['label'] +[i for i in range(data_x_compress.shape[1])])
    print(data_compressed)
    return data_compressed



In [6]:
def generate_split(splits, num_data):
    indx = [ x for x in range(num_data)]
    #print(temp)
    train_indx = list(np.random.choice(indx, size=splits['train'], replace=False))
    temp = [x for x in indx if x not in train_indx]
    #print(len(train_indx))
    # val_indx = list(np.random.choice(temp, size=55, replace=False))
    test_indx = temp
    #print(len(test_indx))
    return train_indx, test_indx

def task_data_split(data, num_task, num_data, splits, random_state=1):
    task_train_byID = {}
    task_test_byID = {}
    task_val_byID = {}
  
    for i in range(num_task):   
        task_data = data.loc[data['label'].eq(i+1)]
        
        sampled_task_data = task_data.sample(n=num_data, random_state=i)
        #print(sampled_task_data.shape, sampled_task_data)
        sampled_data = data.sample(n=num_data, random_state=i)

        train_indx, test_indx = generate_split(splits, num_data)

        task_train_byID[i+1] = np.vstack((sampled_task_data.iloc[train_indx].to_numpy(), \
            sampled_data.iloc[train_indx].to_numpy()))
        task_train_byID[i+1] = np.hstack( (task_train_byID[i+1], np.ones((len(train_indx)*2, 1))) )
        
        temp = task_train_byID[i+1][:, 0]==(i+1)
        task_train_byID[i+1][:, 0][temp==True] = 1.
        task_train_byID[i+1][:, 0][temp==False] = 0.
        print(task_train_byID[i+1].shape, np.sum(task_train_byID[i+1][:,0]))


        task_test_byID[i+1] = np.vstack((sampled_task_data.iloc[test_indx].to_numpy(), \
            sampled_data.iloc[test_indx].to_numpy()))
        task_test_byID[i+1] = np.hstack( (task_test_byID[i+1], np.ones((len(test_indx)*2, 1))) )

        temp = task_test_byID[i+1][:, 0]==(i+1)
        task_test_byID[i+1][:, 0][temp==True] = 1.
        task_test_byID[i+1][:, 0][temp==False] = 0.
        print(task_test_byID[i+1].shape, np.sum(task_test_byID[i+1][:,0]))

    print('tot task ', len(list(task_train_byID.keys())))
    return task_train_byID, task_test_byID


splits = {}
splits['train'] = 40
splits['test'] = 0
task_train_byID, task_test_byID = task_data_split(data_label_feature, num_task, num_data, splits)

(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 41.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 41.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 41.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 41.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050) 40.0
(0, 2050) 0.0
(80, 2050)

In [7]:
#test the coef in logistic regression

def singleTaskTrain(task_train, task_test):
    X = task_train[:, 1:]
    y = task_train[:, 0]

    clf = LogisticRegression(fit_intercept = False, max_iter=100, C=1.0, random_state=0).fit(X, y)
    pred_y = clf.predict(X)

    # X_test = task_test[:, 1:]
    # y_test = task_test[:, 0]
    # pred_y_test = clf.predict(X_test)

    param = clf.coef_
    #print('pred_y_test ', np.sum(pred_y_test == y_test)/y_test.shape[0], param.shape)

    #bias = clf.intercept_
    #return np.hstack((param, np.atleast_2d(bias)))
    return param, 1

weight = singleTaskTrain(task_train_byID[4], task_test_byID[4])

In [8]:
net = FuncRecursiveNet([
    FLinearLayer(1, False)
])

init_param = net.initialize_weights(utils.toTensor(task_train_byID[1][:, 1:]))
p_lst = [p.size() for p in init_param]
print(p_lst)
shape_record = {}
for i , p in enumerate(p_lst):
    shape_record[i] = [tuple(p)]
print(shape_record)

[torch.Size([1, 2049])]
{0: [(1, 2049)]}


In [9]:
def gen_attr(path_attributes):
    data_attributes = pd.read_csv(path_attributes, sep="\n", header=None)
    print(data_attributes)
    a = ppp
    lst = []
    task_attr_byID = {}
    i = 1
    for r in data_attributes.iterrows():
        # print(len(r), )
        s = r[1].to_numpy()
        # print(len(s[0].split()), s[0].split())
        print(len(s[0].split()))
        task_attr_byID[i] = np.array([float(a) for a in s[0].split()])
        print(task_attr_byID[i].shape)
        i += 1
    return task_attr_byID

def gen_attr_zScore(path_attributes):
    data_attributes = pd.read_csv(path_attributes, sep="\n", header=None)
    print(data_attributes)
    lst = []
    task_attr_byID = {}
    i = 1
    for r in data_attributes.iterrows():
        #print(len(r), )
        s = r[1].to_numpy()
        # print(len(s[0].split()), s[0].split())
        #print(r[0], len(s[0].split()))
        lst.append(np.atleast_2d(np.array([float(a) for a in s[0].split()])))
        # print(task_attr_byID[i].shape)
        # i += 1
    attr_mat = np.concatenate(lst, axis=0)
    # print('attr_mat ', attr_mat.shape, attr_mat)
    # attr_mean = np.mean(attr_mat, axis=0)
    # print('attr_mean ', attr_mean.shape, attr_mean)


    # attr_sub_mean = attr_mat - attr_mean
    # print('sub ', attr_sub_mean.shape, attr_mat - attr_sub_mean)

    # attr_std = np.std(attr_mat, axis=0)
    # print('attr_std ', attr_std.shape, attr_std)

    # attr_z = attr_sub_mean/attr_std
    # print('z score', attr_z.shape, attr_sub_mean/attr_z)

    # print('ffff ',attr_z)
    attr_mat_normalize = attr_mat/100
    for t in range(attr_mat.shape[0]):
        # task_attr_byID[t+1] = attr_z[t,:]
        task_attr_byID[t+1] = attr_mat_normalize[t,:]
    
    print(list(task_attr_byID.keys()))
    return task_attr_byID


task_attr_byID = gen_attr_zScore(path_attributes)

0
0    0.0 2.9197080292 1.4598540146 0.0 59.854014598...
1    4.41176470588 4.41176470588 2.94117647059 1.47...
2    0.0 3.97350993377 3.31125827815 0.0 70.8609271...
3    0.0 1.4598540146 10.2189781022 0.0 0.0 0.0 8.7...
4    0.0 0.0 3.22580645161 0.0 1.0752688172 0.0 0.0...
..                                                 ...
195  2.7027027027 12.8378378378 0.0 0.0 0.0 0.0 76....
196  18.3006535948 11.7647058824 0.0 3.26797385621 ...
197  10.7594936709 36.7088607595 0.0 7.59493670886 ...
198  0.0 2.06896551724 0.0 2.06896551724 0.0 0.0 88...
199  0.0 8.28025477707 0.0 0.0 0.0 0.0 82.802547770...

[200 rows x 1 columns]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 9

In [10]:
def ZSTL_train_test_val(num_task, task_train_byID, task_test_byID, task_attr_byID, destination):
    task_train_data = {}
    task_test_data = {}
    task_val_data = {}
    acc = 0
    for t in range(num_task):
        weight, a = singleTaskTrain(task_train_byID[t+1], task_test_byID[t+1])
        acc += a
        cur_task_train = (task_attr_byID[t+1], weight, task_train_byID[t+1][:,1:], np.atleast_2d(task_train_byID[t+1][:,0]).T)
        cur_task_test = (task_attr_byID[t+1], weight, task_test_byID[t+1][:,1:], np.atleast_2d(task_test_byID[t+1][:,0]).T)

        task_train_data[t+1] = cur_task_train
        task_test_data[t+1] = cur_task_test

    print(len(task_train_data))
    print(len(task_test_data))
    
    with open(destination+'task_train_data_normalize.pickle', 'wb') as handle:
        pickle.dump(task_train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(destination+'task_test_data_normalize.pickle', 'wb') as handle:
        pickle.dump(task_test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print('mean acc ', acc/num_task)


# a = {'hello': 'world'}

# with open('filename.pickle', 'wb') as handle:
#     pickle.dump([a], handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('filename.pickle', 'rb') as handle:
#     b = pickle.load(handle)

# print(a == b[0])
ZSTL_train_test_val(num_task, task_train_byID, task_test_byID, task_attr_byID, path_destination)

200
200
mean acc  1.0
